In [1]:
from dm_control import suite
import numpy as np
import os
os.environ["MUJOCO_GL"] = "egl"

# Load the swimmer environment
env = suite.load(domain_name="swimmer", task_name="swimmer6")
t0 = env.reset()

# Step a random action
action_spec = env.action_spec()
random_action = np.random.uniform(action_spec.minimum, action_spec.maximum, action_spec.shape)
time_step = env.step(random_action)

In [2]:
t0.observation

OrderedDict([('joints',
              array([-0.82784066, -0.00798398,  0.1395953 ,  0.55921839,  0.87159999])),
             ('to_target', array([0.39007679, 0.59967893])),
             ('body_velocities',
              array([-0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0.,
                     -0.,  0., -0.,  0.,  0.]))])

In [3]:
time_step.observation

OrderedDict([('joints',
              array([-0.84165039,  0.01071577,  0.13126839,  0.55363996,  0.88246059])),
             ('to_target', array([0.3922609 , 0.59793336])),
             ('body_velocities',
              array([-3.62001046e-03,  1.27486808e-02,  2.38803869e-01,  1.04119581e-02,
                     -3.11033229e-03, -6.05718540e-01,  1.35775766e-02, -3.54625466e-03,
                      5.41692858e-01, -1.54931064e-02, -1.74778189e-03,  3.27614092e-02,
                      2.38570858e-04,  7.52065684e-03, -3.14591806e-01, -1.82363166e-03,
                     -7.55476179e-03,  3.55513712e-01]))])

In [4]:
mjdata = env.physics.data
mjdata.qfrc_passive

array([ 7.72429164e-06, -1.84408561e-04, -7.97853411e-05, -7.57808375e-05,
       -1.10603039e-04, -3.94950486e-05, -3.43043208e-06, -1.23479092e-05])

In [5]:
mjdata.qfrc_fluid

array([ 7.72429164e-06, -1.84408561e-04, -7.97853411e-05, -7.57808375e-05,
       -1.10603039e-04, -3.94950486e-05, -3.43043208e-06, -1.23479092e-05])

In [6]:
mjdata.qfrc_spring

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [7]:
def apply_fluid_drift(physics, to_target, force_magnitude):
    """
    Apply a constant backward force to simulate fluid drift.

    Args:
        physics: The `mjcf.Physics` object or `mjData` instance.
        force_magnitude: Magnitude of the force in the negative x-direction.
    """
    # Define the force vector (negative x-direction)
    force_vector = - force_magnitude * to_target / np.linalg.norm(to_target) 
    force_vector = np.append(force_vector, 0.0)
    
    n_bodies = physics.model.nbody
    force_matrix = np.tile(force_vector, (n_bodies, 1))

    # Apply the cartesian force (fx, fy, fz). 
    physics.data.xfrc_applied[:, :3] = force_matrix  # Apply force


In [8]:
np.tile([-2, 2], (6, 1))


array([[-2,  2],
       [-2,  2],
       [-2,  2],
       [-2,  2],
       [-2,  2],
       [-2,  2]])

In [9]:
physics = env.physics
physics.model.nbody

7

In [10]:
from PIL import Image


# Reset the environment
env.reset()
print(env.observation_spec())
# Define a zero action policy
zero_action = np.zeros(action_spec.shape)
random_action = np.random.uniform(action_spec.minimum, action_spec.maximum, action_spec.shape)
# Step the environment with zero action
time_step = env.step(zero_action)
for _ in range(10):
    time_step = env.step(zero_action)
    to_target = time_step.observation['to_target']
    mjdata = env.physics.data
    apply_fluid_drift(env.physics, to_target, 1)
    print(mjdata.qfrc_fluid, '\n', mjdata.qfrc_applied, '\n', mjdata.xfrc_applied, '\n', time_step.observation['to_target'])
    # Render the environment
    pixels = env.physics.render()
    image = Image.fromarray(pixels)
    display(image)

In [11]:
physics = env.physics
for body_id in range(physics.model.nbody):
    # Apply the force to each body in the world Cartesian frame
    print()

In [12]:
import os, sys
os.chdir('/Users/reecekeller/Documents/neuroagents/zebrafish_agent')
os.environ["MUJOCO_GL"] = "egl"
!pwd


/Users/reecekeller/Documents/neuroagents/zebrafish_agent


In [13]:
from zfa_rl_agent.core.environments.zebrafish import swimmer
env = swimmer()
zero_action = np.zeros(action_spec.shape)
time_step = env.step(zero_action)
for _ in range(10):
    time_step = env.step(zero_action)
    pixels = env.physics.render()
    image = Image.fromarray(pixels)
    display(image)

ValueError: Error: Error opening file '/home/rdkeller/zebrafish_agent/zfa_rl_agent/core/environments/common/OutdoorGrassFloorD.png': No such file or directory